Libraries importing

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn import datasets, ensemble
from sklearn.model_selection import train_test_split
from sklearn.metrics import make_scorer
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import r2_score

Importing dataset loaded on my github

In [ ]:
sample_sub = pd.read_csv('https://raw.githubusercontent.com/AnikaJha/Sas_hw/main/sample_submission.csv')

In [ ]:
df_train = pd.read_csv('https://raw.githubusercontent.com/AnikaJha/Sas_hw/main/train.csv')
df_train.period_start_dt = df_train.period_start_dt.apply(pd.to_datetime)
df_train['month'] = df_train.period_start_dt.apply(lambda x: x.month)
df_train['day_of_year'] = df_train.period_start_dt.apply(lambda x: x.dayofyear)
df_train = df_train.drop(columns =['period_start_dt'])
df_train

,Unnamed: 0,product_rk,store_location_rk,demand,PROMO1_FLAG,PROMO2_FLAG,PRICE_REGULAR,PRICE_AFTER_DISC,NUM_CONSULTANT,AUTORIZATION_FLAG,month,day_of_year
0,0,40369,309,29.0,NaN,NaN,NaN,NaN,NaN,NaN,12,354
1,1,40370,309,64.0,NaN,NaN,NaN,NaN,NaN,NaN,12,354
2,2,40372,309,32.0,NaN,NaN,NaN,NaN,NaN,NaN,12,354
3,3,40373,309,10.0,NaN,NaN,NaN,NaN,NaN,NaN,12,354
4,4,46272,309,15.0,NaN,NaN,NaN,NaN,NaN,NaN,12,354
...,...,...,...,...,...,...,...,...,...,...,...,...
35339,35537,40370,1380,NaN,0.0,0.0,1000.00,1000.0,0.0,1.0,12,364
35340,35538,40372,1380,NaN,0.0,0.0,2000.00,2000.0,0.0,1.0,12,364
35341,35539,40373,1380,NaN,0.0,0.0,3000.00,3000.0,0.0,1.0,12,364
35342,35540,46272,1380,NaN,1.0,0.0,284.29,199.0,0.0,1.0,12,364


In [ ]:
#split on train and test.
df_test = df_train[df_train.demand.isna()].copy()
df_train = df_train[df_train.demand.notna()].copy()

# procces data with filling NaNs
df_train.dropna(subset=['NUM_CONSULTANT', 'AUTORIZATION_FLAG'], inplace=True)
df_train.fillna(df_train.mean(), inplace=True)
df_test.fillna(df_test.mean(), inplace=True)

#check
display((df_train.isna().sum()).sum())
display((df_test.isna().sum()).sum())

0

1200

In [ ]:
#preparing parameters for model training
y = df_train.demand
X = df_train.drop(columns=['demand'])
X.drop(columns=['Unnamed: 0'], inplace=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
# prepare parameters
def smape(y_tr, y_pr):
    return 100/len(y_tr) * np.sum(2 * np.abs(y_pr - y_tr) / (np.abs(y_pr) + np.abs(y_tr)))

mape_scorer = make_scorer(
    smape,
    greater_is_better=False
)

params = {'n_estimators': 600, # changed from 200
          'max_depth': 12, # changed from 6
          'min_samples_split': 2,
          'learning_rate': 0.01,
          'criterion':'mse',
          'min_samples_split': 10,
          'loss': 'ls'}


In [ ]:
#train model
gb_reg = ensemble.GradientBoostingRegressor(**params, random_state=0)
gb_reg.fit(X_train, y_train)

GradientBoostingRegressor(criterion='mse', learning_rate=0.01, loss='ls',
                          max_depth=12, min_samples_split=10, n_estimators=600,
                          random_state=0)

In [ ]:
#predict model
df_test.drop(columns=['demand', 'Unnamed: 0'], inplace=True)

model = ensemble.GradientBoostingRegressor(**params, random_state=0)
model.fit(X, y)

#update column to submimission file
predicts = model.predict(df_test)
predicts= predicts.astype(int)

sample_sub['predicted'] = predicts

sample_sub.to_csv('My_Submission2.csv', index=0)
sample_sub

,id,predicted
0,908,7
1,909,7
2,910,3
3,911,3
4,912,2
...,...,...
1195,35537,132
1196,35538,51
1197,35539,27
1198,35540,11
